# Creating LDA in Python

 In the last section, we learned a lot about the key idea and concepts behind LDA topic modeling. Let's now put those ideas into practice. In this section, we will be using the Gensim library to create our topic model and the PyLDAVis library to visualize it. You can install both libraries with `pip` with the following commands:
 
```
pip install gensim
```

and

```
pip install pyldavis
```

We will also need to install NLTK, or the Natural Language Toolkit, in order to get a list of stop words. We can install the library with pip:

```
pip install nltk
```

Once you have installed `NLTK`, you will need to download the list of English stop words. You can do so with the following command:

```
nltk.download('stopwords')
```

## Importing the Required Libraries and Data

Now that we have our libraries installed correctly, we can import everything.

In [1]:
import pandas as pd
from nltk.corpus import stopwords
import string
import gensim.corpora as corpora
from gensim.models import LdaModel
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

This will import the requisite model from Gensim. For this notebook, we will be using the `LdaModel` class. This class allows us to create an LDA model. Before we can populate our model, however, we must first load and clean our data.

In [2]:
df = pd.read_csv("../data/trc.csv")
df = df[["Last", "First", "Description"]]
df

,Last,First,Description
0,AARON,Thabo Simon,An ANCYL member who was shot and severely inju...
1,ABBOTT,Montaigne,A member of the SADF who was severely injured ...
2,ABRAHAM,Nzaliseko Christopher,A COSAS supporter who was kicked and beaten wi...
3,ABRAHAMS,Achmat Fardiel,Was shot and blinded in one eye by members of ...
4,ABRAHAMS,Annalene Mildred,Was shot and injured by members of the SAP in ...
...,...,...,...
20829,XUZA,Mandla,Was severely injured when he was stoned by a f...
20830,YAKA,Mbangomuni,An IFP supporter and acting induna who was sho...
20831,YALI,Khayalethu,"Was shot by members of the SAP in Lingelihle, ..."
20832,YALO,Bikiwe,An IFP supporter whose house and possessions w...


Our goal in this section will be to model all the descriptions of violence in the TRC Volume 7 Final Report. We will, therefore, grab all documents and place them into a list.

In [3]:
docs = df.Description.tolist()
docs[0]

"An ANCYL member who was shot and severely injured by SAP members at Lephoi, Bethulie, Orange Free State (OFS) on 17 April 1991. Police opened fire on a gathering at an ANC supporter's house following a dispute between two neighbours, one of whom was linked to the ANC and the other to the SAP and a councillor."

## Cleaning Documents

Now that we have our documents, let's go ahead and load up our stop words. These will be the words that we remove from our documents.

In [4]:
stop_words = stopwords.words('english')

Next, we need a function to clean our documents. The purpose of the function below is to take a single document as an input and return a cleaned sequence of words with no punctuation or stop words.

In [5]:
def clean_doc(doc):
    no_punct = ''
    for c in doc:
        if c not in string.punctuation:
            no_punct = no_punct+c
    # with list comprehension
    # no_punct = ''.join([c for c in doc if c not in string.punctuation])
    
    words = no_punct.lower().split()
    
    final_words = []
    for word in words:
        if word not in stop_words:
            final_words.append(word)
    
    # with list comprehension
    # final_words = [word for word in words if word not in stop_words]

    return final_words

Let's take a look and see what this looks like now when we run it over our first document.

In [6]:
cleaned = clean_doc(docs[0])
print(docs[0])
print(cleaned)

An ANCYL member who was shot and severely injured by SAP members at Lephoi, Bethulie, Orange Free State (OFS) on 17 April 1991. Police opened fire on a gathering at an ANC supporter's house following a dispute between two neighbours, one of whom was linked to the ANC and the other to the SAP and a councillor.
['ancyl', 'member', 'shot', 'severely', 'injured', 'sap', 'members', 'lephoi', 'bethulie', 'orange', 'free', 'state', 'ofs', '17', 'april', '1991', 'police', 'opened', 'fire', 'gathering', 'anc', 'supporters', 'house', 'following', 'dispute', 'two', 'neighbours', 'one', 'linked', 'anc', 'sap', 'councillor']


With our function created, we can now process all our documents. In the line be below, we will convert all documents into a list called `cleaned_docs`. This will be our documents that are now represented as a sequence of words.

In [7]:
cleaned_docs = [clean_doc(doc) for doc in docs]

## Create ID-Word Index

Remember, an LDA topic model cannot look at words, rather it must look at numbers in order for the algorithm to work. This means that we need to convert all words into numbers. We can do this with a bag-of-words approach. In this approach, we create an ID-Word Index. This is essentially a dictionary where each unique word has a unique number. The dictionary is sorted alphabetically.

In [8]:
id2word = corpora.Dictionary(cleaned_docs)

With the ID-Word Index created, let's query it and see what word maps to index number `250`.

In [9]:
id2word[250]

'bmw'

As we can see, it is `BMW`, a political organization in South Africa. Now that we have our dictionary, we can convert all our documents into a sequence of numbers, rather than words. We can do this via the `doc2bow()` method. 

In [10]:
corpus = [id2word.doc2bow(cleaned_doc) for cleaned_doc in cleaned_docs]

The `corpus` object now contains all the documents but represented as a bag-of-words index, rather than as a sequence of words. This is the precise data that our LDA model will expect.

In [11]:
print(corpus[0])

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


In order to see what these numbers correspond to, let's take a look at our first document and map each number to the ID-Word Index.

In [12]:
for num in corpus[0]:
    num = num[0]
    print(f"{num}\t{id2word[num]}")

0	17
1	1991
2	anc
3	ancyl
4	april
5	bethulie
6	councillor
7	dispute
8	fire
9	following
10	free
11	gathering
12	house
13	injured
14	lephoi
15	linked
16	member
17	members
18	neighbours
19	ofs
20	one
21	opened
22	orange
23	police
24	sap
25	severely
26	shot
27	state
28	supporters
29	two


## Creating LDA Topic Model

With our `corpus` now prepared,, we can pass it to our LDA Model. We need have a lot of parameters that we can use here, but three things are mandatory:

- `corpus` which will be our corpus object
- `id2word` which will be our ID-Word Index
- `num_topics` which will be the number of topics we want the model to find.

Now, we can train our topic model.

In [13]:
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=100)

Training a model may take several seconds to minutes. The more documents you have, the longer it will take. Once the model is trained, we can begin to analyze the model.

## Analyze a Document

If we want to get the topics for each document in our corpus, we can do so via `get_document_topics()`. This will take a single argument, the `corpus` object. It will return a list of topics and a score for each topic.

In [14]:
topics = lda_model.get_document_topics(corpus)
len(topics)

20834

We can analyze the topics for our first document.

In [15]:
topics[0]

[(6, 0.037855655),
 (15, 0.037333403),
 (27, 0.5222747),
 (36, 0.06073708),
 (43, 0.041372776),
 (74, 0.21932246),
 (94, 0.05292057)]

These topics were assigned to the following document:

In [16]:
print(docs[0])

An ANCYL member who was shot and severely injured by SAP members at Lephoi, Bethulie, Orange Free State (OFS) on 17 April 1991. Police opened fire on a gathering at an ANC supporter's house following a dispute between two neighbours, one of whom was linked to the ANC and the other to the SAP and a councillor.


Let's now print off some of the key words for each of the top-3 topics. For each topic, we will print off the top-10 words with the `get_topic_terms()` method which will take two arguments: the topic number and the amount of words to return.

In [17]:
for topic in topics[0][:3]:
    terms = lda_model.get_topic_terms(topic[0], 10)
    print(topic)
    for num in terms:
        num = num[0]
        print(num, id2word[num])
    print()

(6, 0.038197085)
733 seriously
1394 boipatong
13 injured
809 left
750 massacre
0 17
284 thirteen
68 people
551 five
406 refused

(15, 0.037302956)
625 campaign
3085 soldiers
226 torture
37 transvaal
7 dispute
1350 military
33 landmine
1224 thrown
711 section
2 anc

(27, 0.5223308)
26 shot
8 fire
21 opened
17 members
23 police
24 sap
13 injured
255 march
58 town
61 1990



These results seem acceptable.

## Analyze the Topic Model

It is often difficult to gauge the quality of a topic model by simply looking at raw output. For these reasons, it is useful to be able to study a model in its entirety. We can do this with PyLDAVis, a library that was designed to display in 2-dimensional space the topics of a topic model and the key words associated with each topic.

In [18]:
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)

c:\Users\wma22\anaconda3\envs\textbook\lib\site-packages\sklearn\manifold\_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


In [19]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
64    -0.263505 -0.019382       1        1  4.214569
60    -0.378828 -0.058674       2        1  4.087336
32    -0.311093  0.087178       3        1  3.365182
3     -0.475425 -0.041942       4        1  3.204816
7      0.187479 -0.492747       5        1  2.998725
...         ...       ...     ...      ...       ...
22     0.116352 -0.170869      96        1  0.325247
45     0.336949  0.072965      97        1  0.294095
98     0.444777  0.159917      98        1  0.259635
46    -0.170074 -0.317671      99        1  0.189554
12     0.038489  0.276368     100        1  0.188674

[100 rows x 5 columns], topic_info=           Term          Freq         Total  Category  logprob  loglift
347         ifp   7949.000000   7949.000000   Default  30.0000  30.0000
28   supporters  10675.000000  10675.000000   Default  29.0000  29.0000
505        1994   2512.000000   2512.000000   Default  28.0000  28.0000
994         tvl   1298.000000   1298.000000   Default  27.0000  27.0000
2           anc  11188.000000  11188.000000   Default  26.0000  26.0000
..          ...           ...           ...       ...      ...      ...
2           anc      9.235738  11188.061947  Topic100  -4.5341  -0.8266
13      injured      5.861281   5936.602787  Topic100  -4.9888  -0.6476
29          two      3.980958   2343.026874  Topic100  -5.3756  -0.1048
352   allegedly      3.854998   2549.389728  Topic100  -5.4078  -0.2213
85         dead      4.208761   4723.345846  Topic100  -5.3200  -0.7502

[4666 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
697       3  0.150945             1
697      17  0.009635             1
697      18  0.001606             1
697      23  0.081895             1
697      25  0.043356             1
...     ...       ...           ...
1577     31  0.988717         ‘red’
2855     67  0.877141  ‘sharpeville
3006     84  0.398323   ‘terrorist’
518      74  0.672499      ‘whites’
520       5  0.949699            ’s

[10878 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[65, 61, 33, 4, 8, 73, 60, 6, 76, 39, 53, 41, 9, 56, 77, 2, 50, 28, 7, 66, 64, 18, 90, 55, 87, 48, 43, 67, 36, 27, 17, 100, 26, 15, 11, 51, 40, 14, 75, 91, 68, 37, 98, 96, 35, 38, 16, 3, 71, 21, 79, 83, 1, 49, 86, 19, 89, 24, 31, 34, 70, 80, 22, 52, 59, 94, 58, 5, 93, 10, 97, 54, 62, 92, 45, 12, 72, 44, 78, 69, 42, 32, 81, 85, 63, 20, 25, 84, 30, 74, 95, 82, 57, 29, 88, 23, 46, 99, 47, 13])

Now, we can see all 100 topics plotted in two-dimensional space and we can study the degree to which the words in each topic are unique to that topic when compared with the corpus as as whole.

While useful as a first step to exploring data, we have more powerful ways of clustering documents today. As we will see over the remainder of the chapter, other libraries and approaches that leverage state of the art language models are perhaps better suited to finding and extracting topics from your corpus.